In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



c:\Users\ayush\OneDrive - iiit-b\CollegeAcads\Sem6\MOML\Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=42)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [4]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient
#adaptive weights based on distance

results = []
count_alpha = 100
trials_per_alpha = 10
for i in range(count_alpha+1):
    alpha = i/count_alpha
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    # print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })

acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 15:59:11,824] A new study created in memory with name: no-name-605ac882-4ab1-43d3-94ac-ddaf72033d9e
[I 2025-05-16 15:59:12,000] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 5, 'eta': 0.2217447019493943, 'subsample': 0.959513046438351, 'colsample_bytree': 0.8483196240565537}.
[I 2025-05-16 15:59:12,115] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.09004907057621439, 'subsample': 0.9871871145990009, 'colsample_bytree': 0.7175239561291465}.
[I 2025-05-16 15:59:12,239] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.06771060077837074, 'subsample': 0.7297874019858482, 'colsample_bytree': 0.8231832606368076}.
[I 2025-05-16 15:59:12,346] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.03336961083270768, 'subsample': 0.8337027453520892, 'colsample_bytree': 0.9300676836761609}.
[I 2025-05-1

Pareto-optimal solutions:
Params: {'alpha': 0.0, 'max_depth': 5, 'eta': 0.2217447019493943, 'subsample': 0.959513046438351, 'colsample_bytree': 0.8483196240565537}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01, 'max_depth': 3, 'eta': 0.2756005520004207, 'subsample': 0.7708311507322702, 'colsample_bytree': 0.8032286404427678}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.22, 'max_depth': 6, 'eta': 0.2879250450156225, 'subsample': 0.8158830467969738, 'colsample_bytree': 0.6234818745924056}, Accuracy: 0.6850, DPD: 0.0061
Params: {'alpha': 0.32, 'max_depth': 7, 'eta': 0.25383063022022256, 'subsample': 0.9531125614047744, 'colsample_bytree': 0.9212622856369936}, Accuracy: 0.6936, DPD: 0.0454
Params: {'alpha': 0.57, 'max_depth': 4, 'eta': 0.27496164961308023, 'subsample': 0.8090403100440355, 'colsample_bytree': 0.6342261856314848}, Accuracy: 0.6857, DPD: 0.0358
Params: {'alpha': 0.63, 'max_depth': 7, 'eta': 0.262919401904006, 'subsample': 0.7321925450514479, 'colsample_bytree': 

In [5]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [ ]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
trials_per_alpha = 50
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=0)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(0,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=1)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(1,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])
count_alpha = 20



for i in range(count_alpha):
    print(i)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc)-normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd)-normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = np.sqrt(acc_dist + dpd_dist)
        if dist > max_dist:
            max_dist = dist
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 11:13:19,261] A new study created in memory with name: no-name-9f3b9129-059c-4f8e-a533-5ccb0b9fddc4
[I 2025-05-16 11:13:19,357] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}.
[I 2025-05-16 11:13:19,460] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.04040577032334644, 'subsample': 0.7488904515273311, 'colsample_bytree': 0.9586222733019236}.
[I 2025-05-16 11:13:19,567] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 3, 'eta': 0.14583696499170543, 'subsample': 0.795810321715773, 'colsample_bytree': 0.952224552097848}.
[I 2025-05-16 11:13:19,666] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.13623371217287475, 'subsample': 0.6081598975377428, 'colsample_bytree': 0.7401898808501025}.
[I 2025-05-1

Completed study for alpha=0


[I 2025-05-16 11:13:25,117] Trial 1 finished with values: [0.7021379578657795, 0.06314616230362322] and parameters: {'max_depth': 6, 'eta': 0.26589168923854556, 'subsample': 0.7897562282613623, 'colsample_bytree': 0.9600111848863682}.
[I 2025-05-16 11:13:25,260] Trial 2 finished with values: [0.6764374312646085, 0.045053618695635365] and parameters: {'max_depth': 4, 'eta': 0.06306784136966327, 'subsample': 0.74798241185463, 'colsample_bytree': 0.6551612112285078}.
[I 2025-05-16 11:13:25,425] Trial 3 finished with values: [0.6978491757879871, 0.08431029941171075] and parameters: {'max_depth': 6, 'eta': 0.18242492595888532, 'subsample': 0.968494180092288, 'colsample_bytree': 0.9277864888887004}.
[I 2025-05-16 11:13:25,594] Trial 4 finished with values: [0.692148159040293, 0.04711257309011945] and parameters: {'max_depth': 6, 'eta': 0.15633414111361582, 'subsample': 0.7677602996238255, 'colsample_bytree': 0.6515532718065272}.
[I 2025-05-16 11:13:25,738] Trial 5 finished with values: [0.69

Completed study for alpha=1
[{'accuracy': 0.5000061268315397, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}}, {'accuracy': 0.7214129698896864, 'dpd': 0.10581076448863501, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 7, 'eta': 0.27926006398945585, 'subsample': 0.8218636333494963, 'colsample_bytree': 0.8238601358476214}}]
0


[I 2025-05-16 11:13:32,507] Trial 1 finished with values: [0.6742945719335974, 0.029715368588148706] and parameters: {'max_depth': 3, 'eta': 0.2624127443857211, 'subsample': 0.6398351480254919, 'colsample_bytree': 0.9273198177738456}.
[I 2025-05-16 11:13:32,593] Trial 2 finished with values: [0.6742976353493672, 0.057803156772305846] and parameters: {'max_depth': 3, 'eta': 0.28080783237841106, 'subsample': 0.6181718122808142, 'colsample_bytree': 0.837916413058396}.
[I 2025-05-16 11:13:32,701] Trial 3 finished with values: [0.666432315360273, 0.04994182214162388] and parameters: {'max_depth': 4, 'eta': 0.1705686695540982, 'subsample': 0.9161626847024184, 'colsample_bytree': 0.8485905178665168}.
[I 2025-05-16 11:13:32,839] Trial 4 finished with values: [0.6828660092576424, 0.045312916176632] and parameters: {'max_depth': 7, 'eta': 0.1203245017352115, 'subsample': 0.794519582525799, 'colsample_bytree': 0.6548483030696385}.
[I 2025-05-16 11:13:32,944] Trial 5 finished with values: [0.67500

...

Completed study for alpha=0.02252197265625
18


[I 2025-05-16 11:15:29,313] Trial 1 finished with values: [0.6657261980253222, 0.016923952882237265] and parameters: {'max_depth': 7, 'eta': 0.2962557453904555, 'subsample': 0.8505436791396601, 'colsample_bytree': 0.9185126430559836}.
[I 2025-05-16 11:15:29,423] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 4, 'eta': 0.05675002143412593, 'subsample': 0.8794169283264468, 'colsample_bytree': 0.851588780909592}.
[I 2025-05-16 11:15:29,543] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 7, 'eta': 0.05391397292806195, 'subsample': 0.9397143497197321, 'colsample_bytree': 0.660788860795329}.
[I 2025-05-16 11:15:29,653] Trial 4 finished with values: [0.6692966091050844, 0.022221463950426807] and parameters: {'max_depth': 6, 'eta': 0.28499925848496943, 'subsample': 0.6676799485288464, 'colsample_bytree': 0.6757276737239967}.
[I 2025-05-16 11:15:29,754] Trial 5 finished with values: [

Completed study for alpha=0.022491455078125
19


[I 2025-05-16 11:15:35,284] Trial 1 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 4, 'eta': 0.23403115822266618, 'subsample': 0.7981434372239434, 'colsample_bytree': 0.8778856782504978}.
[I 2025-05-16 11:15:35,405] Trial 2 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 3, 'eta': 0.15653418706676264, 'subsample': 0.7115916132762418, 'colsample_bytree': 0.8443908848876551}.
[I 2025-05-16 11:15:35,544] Trial 3 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.03432410931468004, 'subsample': 0.9225001339521833, 'colsample_bytree': 0.923817762955518}.
[I 2025-05-16 11:15:35,646] Trial 4 finished with values: [0.6707226291459504, 0.012983203043084202] and parameters: {'max_depth': 5, 'eta': 0.16109160570667802, 'subsample': 0.7853296256998328, 'colsample_bytree': 0.7854917304418363}.
[I 2025-05-16 11:15:35,754] Trial 5 finished with values:

Completed study for alpha=0.0224761962890625
Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 6, 'eta': 0.13110633265877236, 'subsample': 0.8125360197138709, 'colsample_bytree': 0.7772282448273781}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.015625, 'max_depth': 5, 'eta': 0.05282487573863137, 'subsample': 0.8688659694886003, 'colsample_bytree': 0.6687411708124102}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01953125, 'max_depth': 5, 'eta': 0.0504980502577411, 'subsample': 0.6485748884994879, 'colsample_bytree': 0.6209503294456052}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021484375, 'max_depth': 3, 'eta': 0.056966858952711155, 'subsample': 0.6032123055973961, 'colsample_bytree': 0.7679043893131532}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.02197265625, 'max_depth': 5, 'eta': 0.19607347343993498, 'subsample': 0.6032555369919693, 'colsample_bytree': 0.734748734600021}, Accuracy: 0.5522, DPD: 0.0037
Params: {'alpha': 0.0224609375, 'max_depth': 3,

In [8]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [ ]:

# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
trials_per_alpha = 50
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=0)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(0,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'])
objective = create_objective_function(alpha=1)  # Example alpha value
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(1,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])
count_alpha = 20



for j in range(count_alpha):
    print(j)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc)-normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd)-normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = np.sqrt(acc_dist + dpd_dist)
        if dist > max_dist:
            max_dist = dist
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'])
    objective = create_objective_function(alpha=alpha)  # Example alpha value
    study.optimize(objective, n_trials=trials_per_alpha)
    print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 16:29:02,867] A new study created in memory with name: no-name-0b5c242b-4fa4-4a68-bbc0-3d1392a5a2de
[I 2025-05-16 16:29:02,991] Trial 0 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 7, 'eta': 0.27116171617649165, 'subsample': 0.7269463254606328, 'colsample_bytree': 0.9728224999470233}.
[I 2025-05-16 16:29:03,113] Trial 1 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.23294846830639912, 'subsample': 0.8121129252839423, 'colsample_bytree': 0.9862355887827093}.
[I 2025-05-16 16:29:03,202] Trial 2 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 4, 'eta': 0.26031430769402997, 'subsample': 0.6271539098071383, 'colsample_bytree': 0.6450638768079656}.
[I 2025-05-16 16:29:03,310] Trial 3 finished with values: [0.5000061268315397, 0.0] and parameters: {'max_depth': 6, 'eta': 0.24993025012366507, 'subsample': 0.7081928200314112, 'colsample_bytree': 0.6796003471219101}.
[I 2025-05

Completed study for alpha=0


[I 2025-05-16 16:29:08,743] Trial 1 finished with values: [0.7000119473215024, 0.07898193303403796] and parameters: {'max_depth': 5, 'eta': 0.2341255213352826, 'subsample': 0.7362992436220663, 'colsample_bytree': 0.7523469733667653}.
[I 2025-05-16 16:29:08,883] Trial 2 finished with values: [0.6828537555945631, 0.06764696523587017] and parameters: {'max_depth': 4, 'eta': 0.2506698200547449, 'subsample': 0.8086727472974691, 'colsample_bytree': 0.7178467064594145}.
[I 2025-05-16 16:29:08,988] Trial 3 finished with values: [0.6692996725208542, 0.04817756105748499] and parameters: {'max_depth': 3, 'eta': 0.2085092550244008, 'subsample': 0.7781783754627386, 'colsample_bytree': 0.8088941231565413}.
[I 2025-05-16 16:29:09,145] Trial 4 finished with values: [0.7121323518149207, 0.11310194278244023] and parameters: {'max_depth': 7, 'eta': 0.2591082674043036, 'subsample': 0.6213487678809081, 'colsample_bytree': 0.7922651751069175}.
[I 2025-05-16 16:29:09,322] Trial 5 finished with values: [0.704

Completed study for alpha=1
[{'accuracy': 0.5000061268315397, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 7, 'eta': 0.27116171617649165, 'subsample': 0.7269463254606328, 'colsample_bytree': 0.9728224999470233}}, {'accuracy': 0.6614404793632996, 'dpd': 0.02108973425244622, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 3, 'eta': 0.12405963726342151, 'subsample': 0.988569064147095, 'colsample_bytree': 0.9609661355808808}}]
0


[I 2025-05-16 16:29:16,307] Trial 1 finished with values: [0.6700149801031147, 0.03769900950832727] and parameters: {'max_depth': 3, 'eta': 0.21993662765015007, 'subsample': 0.7073724097825345, 'colsample_bytree': 0.7230577635579022}.
[I 2025-05-16 16:29:16,516] Trial 2 finished with values: [0.6693012042287392, 0.03854497802997558] and parameters: {'max_depth': 4, 'eta': 0.03253760214907243, 'subsample': 0.658715987919677, 'colsample_bytree': 0.9058659661805266}.
[I 2025-05-16 16:29:16,667] Trial 3 finished with values: [0.6849935515098045, 0.044985769413101784] and parameters: {'max_depth': 7, 'eta': 0.17212068410182302, 'subsample': 0.9734395485604017, 'colsample_bytree': 0.6864049830524811}.
[I 2025-05-16 16:29:16,801] Trial 4 finished with values: [0.6828706043812972, 0.05656055547910809] and parameters: {'max_depth': 4, 'eta': 0.24491174936806678, 'subsample': 0.8110930651722075, 'colsample_bytree': 0.7921512131361431}.
[I 2025-05-16 16:29:16,920] Trial 5 finished with values: [0

KeyboardInterrupt: 

In [7]:
# Create interactive Plotly plot
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [8]:
plot_df

,DPD,Accuracy,alpha
0,0.000000,0.500006,0.000000
1,0.000000,0.500006,0.015625
2,0.000041,0.505013,0.019531
3,0.002757,0.557231,0.021484
4,0.002757,0.557231,0.021973
5,0.000000,0.500006,0.022095
6,0.003839,0.578633,0.022163
7,0.008641,0.672869,0.022171
8,0.008641,0.672869,0.022217
9,0.008641,0.672869,0.022461
